In [5]:
import mvtec
import train
import vanila_ae as ae

In [15]:
DATA_DIR= '/mnt/d/datasets/mvtec'
IMG_SIZE = 256
NORMALIZE = False
CATEGORY = 'bottle'
BATCH_SIZE = 32
VALID_RATIO = 0.2

train_transform, test_transform = mvtec.get_transforms(
    img_size=IMG_SIZE, 
    normalize=NORMALIZE,
)
train_loader, valid_loader, test_loader = mvtec.get_dataloaders(
    data_dir=DATA_DIR,
    category=CATEGORY,
    batch_size=BATCH_SIZE,
    valid_ratio=VALID_RATIO,
    train_transform=train_transform,
    test_transform=test_transform,
)

Loaded bottle train: 209 images
>> Normal:  209
>> Anomaly: 0
Loaded bottle test: 83 images
>> Normal:  20
>> Anomaly: 63
Loaded bottle train: 209 images
>> Normal:  209
>> Anomaly: 0


In [16]:
IN_CHANNESL = 3
OUT_CHANNELS = 3
LATENT_DIM = 512

device = train.get_device()

model = ae.VanillaAutoEncoder(
    in_channels=IN_CHANNESL, 
    out_channels=OUT_CHANNELS, 
    latent_dim=LATENT_DIM).to(device)

>> Random Seed: 42
>> Device: cuda
>> CUDA available: True
>> GPU: NVIDIA GeForce GTX 1080 Ti


In [17]:
train.quick_train_test(model, train_loader, valid_loader, test_loader,
                       device=device,
                       epochs=10)

/home/nampl/anaconda3/envs/pytorch_env/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


빠른 훈련 및 테스트 시작
모델 파라미터 수: 30,979,715
Starting training...


✓ New best model saved (Valid Loss: 0.4538)
Epoch  1/10 | Train: Loss=0.3391, PSNR=8.89, SSIM=0.176 | Valid: Loss=0.4538, PSNR=5.88, SSIM=0.090 | Time: 6.7s


✓ New best model saved (Valid Loss: 0.3301)
Epoch  2/10 | Train: Loss=0.2636, PSNR=9.54, SSIM=0.381 | Valid: Loss=0.3301, PSNR=8.15, SSIM=0.257 | Time: 6.2s


Epoch  3/10 | Train: Loss=0.2545, PSNR=9.56, SSIM=0.410 | Valid: Loss=0.4452, PSNR=5.47, SSIM=0.178 | Time: 3.5s


...

✓ New best model saved (Valid Loss: 0.2241)
Epoch  9/10 | Train: Loss=0.2001, PSNR=10.99, SSIM=0.519 | Valid: Loss=0.2241, PSNR=10.50, SSIM=0.461 | Time: 6.3s


Epoch 10/10 | Train: Loss=0.1971, PSNR=11.09, SSIM=0.524 | Valid: Loss=0.2574, PSNR=10.79, SSIM=0.337 | Time: 6.2s
Training completed. Best validation loss: 0.2241

빠른 평가 중...
Evaluating on test set...


Testing: 100%|██████████| 3/3 [00:02<00:00,  1.10it/s]



평가 결과:
  AUROC: 0.7341
  AUPR: 0.9203
  Accuracy: 0.7108
  Precision: 0.9756
  Recall: 0.6349
  F1-Score: 0.7692
  Threshold: 0.08960685133934021
  Normal_Score_Mean: 0.08897687494754791
  Normal_Score_Std: 0.0005575212417170405
  Anomaly_Score_Mean: 0.09073308110237122


(VanillaAutoEncoder(
   (encoder): VanillaEncoder(
     (conv_blocks): Sequential(
       (0): ConvBlock(
         (conv_block): Sequential(
           (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
           (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
           (2): LeakyReLU(negative_slope=0.2, inplace=True)
         )
       )
       (1): ConvBlock(
         (conv_block): Sequential(
           (0): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
           (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
           (2): LeakyReLU(negative_slope=0.2, inplace=True)
         )
       )
       (2): ConvBlock(
         (conv_block): Sequential(
           (0): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
           (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
           (2): LeakyReLU(negative_slope=0.2,